# The Solow Model with Land

In this section of the code, we import the required libraries and Python scripts.

In [17]:
import numpy as np
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Python script
from modelproject import solow_equation, multi_start
from modelproject import solow_equation1, multi_start1

# 1. Model description

We use Chapter 7.1 in "xxx" as our baseline model, which highlights the importance of a natural resource by emphasizing the role of land where we have the following equations:

\begin{aligned}
    Y_{t} &= K^{\alpha}_{t}(A_{t}L_{t})^{\beta}X^{\kappa}, \quad \alpha>0, \beta>0, \kappa>0, \alpha+\beta+\kappa=1 \hspace{4em} &(1)\\
    K_{t+1} &= sY_{t} + (1-\delta)K_{t}, \quad 0 < s < 1, 0 < \delta < 1, \quad K_{0} \text{ given}\hspace{4em} &(2)\\
    L_{t+1} &= (1+n)L_{t}, \quad n≥0,  \quad L_{0} \text{ given}\hspace{4em} &(3)\\
    A_{t+1} &= (1+g)A_{t}, \quad g≥0, \quad A_{0}  \text{ given} \hspace{4em} &(4)
\end{aligned}

1. The first equation is a Cobb-Douglas aggregate production function with land and exogenous constant technical coefficients $\alpha$, $\beta$ and $\kappa$. The input X of land does not carry a time subscript since land is in fixed supply. <br>
2. The second equation is the capital accumulation equation, assuming given rates of gross investments in capital and of depreciation. <br>
3. Equation (3) and (4) assume given growth rates for the labour force, $L_{t}$, and for the labour-augmenting productivity factor, $A_{t}$. Thus n is the growth rate of the labour force, and g reflects the rate of labour-augmenting technological progress. We assume both of these rates are at least zero. <br>

We asumme constant return to scale for $K_{t}$, $L_{t}$ and X. When X is constant over time there is diminishing returns to scale for $K_{t}$ and $L_{t}$.


## 1.2 Income shares of production factors

In [18]:
# This line creates symbolic representation for various variables and parameters
Y_t, K_t, X, A_t, L_t, alpha, beta, kappa, r_t, w_t, landrent_t = sm.symbols('Y_t, K_t, X, A_t, L_t, alpha, beta, kappa, r_t, w_t, landrent_t')

# This line defines the production function
production_function = K_t**alpha * (A_t*L_t)**beta * X**kappa

# These lines computes the partial derivative of the production function respect to capital, labour, and, land to find the rent of the capital, the wages, and, the rent of the land.
rent = sm.diff(production_function, K_t)
wages = sm.diff(production_function, L_t)
landrent = sm.diff(production_function, X)

# To show the equations for rent, wages and landrent, we create them as an equation:
display_rent = sm.Eq(r_t, rent)
display_wages = sm.Eq(w_t, wages)
display_landrent = sm.Eq(landrent_t, landrent)

# Command for displaying in SymPy format
display(display_rent, display_wages, display_landrent)

Eq(r_t, K_t**alpha*X**kappa*alpha*(A_t*L_t)**beta/K_t)

Eq(w_t, K_t**alpha*X**kappa*beta*(A_t*L_t)**beta/L_t)

Eq(landrent_t, K_t**alpha*X**kappa*kappa*(A_t*L_t)**beta/X)

In [19]:
# Then we would like to substistute the production function with Y_t
rent = rent.subs(production_function, Y_t)
wages = wages.subs(production_function, Y_t)
landrent = landrent.subs(production_function, Y_t)

# Again, we create equations for rent, wages and landrent:
display_rent = sm.Eq(r_t, rent)
display_wages = sm.Eq(w_t, wages)
display_landrent = sm.Eq(landrent_t, landrent)

# Display rent and wages in SymPy format
display(display_rent, display_wages, display_landrent)

Eq(r_t, Y_t*alpha/K_t)

Eq(w_t, Y_t*beta/L_t)

Eq(landrent_t, Y_t*kappa/X)

We have now shown that the income shares of capital, labour and land ($r_{t}K_{t}/Y_{t}$, etc.) are $\alpha$, $\beta$ and $\kappa$, respectively, so the model is one of constant income shares.

# 2 Solving the Steady-State

The Solow model with land can be analyzed directly in terms of the capital-output ratio, $z_{t}≡K_{t}/Y_{t}=k_{t}/y_{t}$, for which we want to establish convergence to a constant steady state level. We will start by using the transition equation for $z_{t}$:
\begin{aligned}
    z_{t+1} &= (\frac{1}{(1+n)(1+g)})^{\beta}[s+(1-\delta)z_{t}]^{1-\alpha}z_{t}^{a} \hspace{4em} &(8)
\end{aligned}

In the Steady-State we have that $z_{t_+1}=z_{t}=z^{*}$. We now isolate $z$:

In [20]:
# Defining parameters
s_val = 0.2
n_val = 0.005
g_val = 0.02
delta_val = 0.06
alpha_val = 0.2
beta_val = 0.6
kappa_val = 0.2
A_0 = 1
L_0 =1

In [21]:
# Call the Multi-Start function to find the steady state value of z.
steady_state_z, smallest_residual = multi_start(num_guesses=100,
            bounds = [1e-5, 50],
            fun = solow_equation, 
            args = (s_val, n_val, g_val, delta_val, alpha_val, beta_val, kappa_val), 
            method = 'hybr')

print("The unique SS solution value for the capital-output ratio is",steady_state_z)

The unique SS solution value for the capital-output ratio is [2.53914905]



The steady state solution is calculated based on parameter values that approximate empirical data.

## 2.1 Graphical solution

In this section, we graphically demonstrate how the transition equation for the capital-output ratio, 𝑧, converges to a specific and constant steady state level, 𝑧∗ Additionally, we visually illustrate the properties of the transition curve: it passes through the origin (0,0), is strictly increasing everywhere, and intersects the 45-degree line at exactly one point where it is strictly positive.

In [22]:
#This function defines the transition equation 
def equation(beta, alpha, delta, n, g, s, z):
    z_next = ((1 / ((1 + n) * (1 + g))) ** beta) * (s + (1 - delta) * z) ** (1 - alpha) * z ** alpha
    return z_next

def plot_graph(beta, alpha, delta, n, g, s):
    z = np.linspace(0, 4, 100)
    z_next = equation(beta, alpha, delta, n, g, s, z)
    
    plt.figure(figsize=(8, 6))
    
    # Plots the transition equation and the 45-degree line
    plt.plot(z, z_next, label='Transition equation')
    plt.plot(z, z, 'r--', label='45-degree line')
    
    # Writing plot aesthetic such as labels, title, legend, and axes limits.
    plt.xlabel('$z_{t}$')
    plt.ylabel('$z_{t+1}$')
    plt.title('Transition equation vs 45-degree line')
    plt.legend()
    plt.xlim(0, 4)
    plt.ylim(0, 4)
    
    plt.show()

# Define sliders with default values
beta_slider = FloatSlider(min=0, max=2, step=0.1, value=0.6, description='beta')
alpha_slider = FloatSlider(min=0, max=1, step=0.1, value=0.2, description='alpha')
delta_slider = FloatSlider(min=0, max=1, step=0.01, value=0.06, description='delta')
n_slider = FloatSlider(min=0, max=1, step=0.001, value=0.005, description='n')
g_slider = FloatSlider(min=0, max=1, step=0.01, value=0.02, description='g')
s_slider = FloatSlider(min=0, max=1, step=0.01, value=0.2, description='s')

# Create interactive plot
interact(plot_graph, beta=beta_slider, alpha=alpha_slider, delta=delta_slider, 
         n=n_slider, g=g_slider, s=s_slider);


interactive(children=(FloatSlider(value=0.6, description='beta', max=2.0), FloatSlider(value=0.2, description=…

The provided code sets up an interactive plot to visually explore the behavior of a transition equation for the capital-output ratio.

# 3 Adding Exhaustible Natural Resources

Land is not the only significant natural resource in aggregate production. Other crucial nonrenewable resources include oil, gas, coal, metals, and non-metallic minerals. Unlike land, whose quantity remains constant during production, non-renewable resources diminish as they are utilized. This depletion implies a greater barrier to growth compared to land. To explore this hypothesis, we plan to expand our Solow model, which currently includes land, to also account for exhaustible natural resources. We denote the total stock by R which each period will be reduced by the amount of E.

The model with land and exhustible natural resources will then be: 

\begin{aligned}
    Y_{t} &= K^{\alpha}_{t}(A_{t}L_{t})^{\beta}X^{\kappa}E_t^{\varepsilon}, \quad \alpha>0, \beta>0, \kappa>0, \varepsilon>0, \alpha+\beta+\kappa+ \varepsilon=1 \hspace{4em} &(1)\\
    K_{t+1} &= sY_{t} + (1-\delta)K_{t}, \quad 0 < s < 1, 0 < \delta < 1, \quad K_{0} \text{ given}\hspace{4em} &(2)\\
    L_{t+1} &= (1+n)L_{t}, \quad n≥0,  \quad L_{0} \text{ given}\hspace{4em} &(3)\\
    A_{t+1} &= (1+g)A_{t}, \quad g≥0, \quad A_{0}  \text{ given} \hspace{4em} &(4)\\
    R_{t+1} &= R_t-E_t\\
    E_t &= \psi R_t \quad 0<\psi<1
\end{aligned}

Adding this to model
1. We enhance the Cobb-Douglas aggregate production function, which already includes land, by adding the variable, E, representing the amount of natural resources utilized.  <br>
4. The depletion of the natural resource, detailing the remaining stock, $R_{t+1}$ , after the natural resources are consumed in production at time, t. <br>
5. The consumption rate of the natural resource in production, which is a constant fraction of the remaining stock each period. <br>


## 3.1 Solving SS

Using the same methoed to calculate the SS for the new transition equation: 

\begin{aligned}
    j_{t+1} &= \frac{1}{((1+n)(1+g))^{\beta}(1-\psi)^\epsilon}[s+(1-\delta)j_{t}]^{1-\alpha}j_{t}^{a} \hspace{4em} &(8)
\end{aligned}

Naming the capital-output ratio j in this version with exhaustible natural resources. 

In [23]:
# Defining parameters
psi_val = 0.05
epsilon_val = 0.1

In [24]:
# Call the Multi-Start1 function to find the steady state value of j.
steady_state_j, smallest_residual = multi_start1(num_guesses=100,
            bounds = [1e-5, 50],
            fun = solow_equation1, 
            args = (s_val, n_val, g_val, delta_val, alpha_val, beta_val, kappa_val, psi_val, epsilon_val), 
            method = 'hybr')

print("The unique SS solution value for the capital-output ratio is",steady_state_j)

The unique SS solution value for the capital-output ratio is [2.33925535]
